<a href="https://colab.research.google.com/github/nhwagle/code-match/blob/main/TransCoder_Based_Code_Functions_to_Embeddings_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is a bit messier, but it also contains content we utilized to process functions into embedding representations utilizing Facebook AI Research's TransCoder models.

### Imports

Installing codegen. Their install_env.sh won't work for colab so modifying such that all conda imports are done with pip

In [ ]:
!git clone https://github.com/facebookresearch/CodeGen.git

Cloning into 'CodeGen'...
remote: Enumerating objects: 2770, done.
remote: Total 2770 (delta 0), reused 0 (delta 0), pack-reused 2770
Receiving objects: 100% (2770/2770), 12.43 MiB | 29.06 MiB/s, done.
Resolving deltas: 100% (1371/1371), done.


In [ ]:
!mv CodeGen/* ./

In [ ]:
import torch
import torchvision
import torchaudio

In [ ]:
%pip install stringcase transformers ply slimit astunparse submitit

     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 49 kB 5.0 MB/s 
     |████████████████████████████████| 88 kB 6.2 MB/s 
     |████████████████████████████████| 71 kB 168 kB/s 
     |████████████████████████████████| 895 kB 50.7 MB/s 
     |████████████████████████████████| 6.6 MB 34.2 MB/s 
     |████████████████████████████████| 596 kB 40.5 MB/s 
     |████████████████████████████████| 77 kB 5.7 MB/s 
  Created wheel for stringcase: filename=stringcase-1.2.0-py3-none-any.whl size=3587 sha256=eda9832bd43267abe82db3cae2e1705d13094b3cf61c3eaf0eb0fb979ad0c17a
  Stored in directory: /root/.cache/pip/wheels/86/ab/a3/a8fa7e0a07e80f547e03468c03827f8257f7339327986faed1
  Created wheel for slimit: filename=slimit-0.8.1-py3-none-any.whl size=79458 sha256=4d72470edd572dd02a2d8e46ec90120f55800605e0035020ea05cf219cc19ecc
  Stored in directory: /root/.cache/pip/wheels/32/0b/89/cee8d5525ed7fa925e6d6ab76a19176c499fd1992c2f822644
Successfully built strin

In [ ]:
import stringcase
import six
import sklearn
import transformers
import ply
import slimit 
import astunparse 
import submitit

In [ ]:
!git clone https://github.com/NVIDIA/apex
%cd apex
%pip install -v --no-cache-dir ./

Cloning into 'apex'...
remote: Enumerating objects: 9614, done.
remote: Counting objects: 100% (685/685), done.
remote: Compressing objects: 100% (349/349), done.
remote: Total 9614 (delta 412), reused 512 (delta 317), pack-reused 8929
Receiving objects: 100% (9614/9614), 14.82 MiB | 26.57 MiB/s, done.
Resolving deltas: 100% (6525/6525), done.
/content/apex
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.headers does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: 

In [ ]:
%cd ..

/content


In [ ]:
%%shell

# pip install cython
# cd codegen_sources/model/tools
# git clone https://github.com/glample/fastBPE.git

# cd fastBPE
# g++ -std=c++11 -pthread -O3 fastBPE/main.cc -IfastBPE -o fast
# python setup.py install
# cd ../../../../

mkdir tree-sitter
cd tree-sitter
git clone https://github.com/tree-sitter/tree-sitter-cpp.git
git clone https://github.com/tree-sitter/tree-sitter-java.git
git clone https://github.com/tree-sitter/tree-sitter-python.git
cd ..

cd codegen_sources/test_generation/
wget https://github.com/EvoSuite/evosuite/releases/download/v1.1.0/evosuite-1.1.0.jar
cd ../..



Cloning into 'tree-sitter-cpp'...
remote: Enumerating objects: 1736, done.
remote: Counting objects: 100% (647/647), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 1736 (delta 620), reused 586 (delta 586), pack-reused 1089
Receiving objects: 100% (1736/1736), 51.47 MiB | 31.02 MiB/s, done.
Resolving deltas: 100% (1145/1145), done.
Cloning into 'tree-sitter-java'...
remote: Enumerating objects: 1798, done.
remote: Counting objects: 100% (407/407), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 1798 (delta 314), reused 365 (delta 284), pack-reused 1391
Receiving objects: 100% (1798/1798), 14.03 MiB | 14.80 MiB/s, done.
Resolving deltas: 100% (1103/1103), done.
Cloning into 'tree-sitter-python'...
remote: Enumerating objects: 2502, done.
remote: Counting objects: 100% (246/246), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 2502 (delta 139), reused 234 (delta 132), pack-reused 2256
Receiving objects: 100% (2502/2502), 

In [ ]:
%pip install sacrebleu=="1.2.11" javalang tree_sitter psutil fastBPE
%pip install hydra-core --upgrade --pre
%pip install black==19.10b0

     |████████████████████████████████| 116 kB 7.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 78 kB 6.9 MB/s 
  Created wheel for sacrebleu: filename=sacrebleu-1.2.11-py3-none-any.whl size=18639 sha256=78820eb55094691913df842aab7b2bd3ac2d367cec8467c02a4e464edcd3286d
  Stored in directory: /root/.cache/pip/wheels/e2/67/3e/e5459f232d19e16b12df98130d7799af72253945c114da1487
  Created wheel for tree-sitter: filename=tree_sitter-0.20.0-cp37-cp37m-linux_x86_64.whl size=304848 sha256=38210f50fe9caefd80f5b40e3cd0b0f7097fb60e3eb35aae0a547db3ea0a6f5a
  Stored in directory: /root/.cache/pip/wheels/73/70/97/43e08c95ce68348303fd3f71adfa96193db8d9cf60a14163e0
  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp37-cp37m-linux_x86_64.whl size=483176 sha256=4422e62a8e90865521373a2062d4b72f38691c96e3256cb3b393a6353cf9358e
  Stored in directory: /root/.cache/pip/wheels/bd/d4/

     |████████████████████████████████| 150 kB 5.2 MB/s 
     |████████████████████████████████| 112 kB 64.2 MB/s 
     |████████████████████████████████| 77 kB 5.4 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=ec54f1786e8c68f944e7786f02630952ecb258000eafbd57e76e5fbeaef45699
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime


     |████████████████████████████████| 97 kB 3.4 MB/s 
     |████████████████████████████████| 843 kB 35.4 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!ls /content/gdrive/MyDrive/RSS/Data

algorithms.csv	code.csv	  code-small.csv  func-rep-small.csv
code-all.csv	code-func-rep.pt  equations.csv   func-rep-small.pt


In [ ]:
## Data Handling and Plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
eqns = pd.read_csv("/content/gdrive/MyDrive/RSS/Data/equations.csv")
algs = pd.read_csv("/content/gdrive/MyDrive/RSS/Data/algorithms.csv")
code = pd.read_csv("/content/gdrive/MyDrive/RSS/Data/code.csv")

In [ ]:
demo_c = code.iloc[1]['code']

In [ ]:
demo_c

'import pdb\nimport decimal\nfrom torch.autograd import Variable\n\ndef check_config_used(config, target_source):\n    config_count = dict(vars(config))\n    #Initialize each key have zero value\n    for k in config_count.keys():\n        config_count[k] = 0\n\n    for source in target_source:\n        fp = open(source, \'r\')\n        text = fp.read()\n        for k in config_count.keys():\n            if(text.find(k) >= 0):\n                config_count[k] = 1\n        fp.close()\n\n    config_unused = []\n\n    #Check whether unused config exists\n    for k in config_count.keys():\n        if(config_count[k] == 0):\n            config_unused.append(k)\n\n    print(\'unused config = \')\n    print(config_unused)\n    assert(len(config_unused) == 0), \'unused config exists, please properly use it or comment it\'\n\n\ndef to_np(x):\n    return x.data.cpu().numpy()\n\n\nclass AverageMeter(object):\n    """Computes and stores the average and current value"""\n\n    def __init__(self):\n 

Modified translate function to only yield embedding

In [ ]:

# Copyright (c) 2019-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#

import os
import argparse
from pathlib import Path
import sys
import torch
from codegen_sources.model.src.logger import create_logger
from codegen_sources.preprocessing.lang_processors.cpp_processor import CppProcessor
from codegen_sources.preprocessing.lang_processors.java_processor import JavaProcessor
from codegen_sources.preprocessing.lang_processors.python_processor import (
    PythonProcessor,
)
from codegen_sources.preprocessing.lang_processors.lang_processor import LangProcessor
from codegen_sources.preprocessing.bpe_modes.fast_bpe_mode import FastBPEMode
from codegen_sources.preprocessing.bpe_modes.roberta_bpe_mode import RobertaBPEMode
from codegen_sources.model.src.data.dictionary import (
    Dictionary,
    BOS_WORD,
    EOS_WORD,
    PAD_WORD,
    UNK_WORD,
    MASK_WORD,
)
from codegen_sources.model.src.utils import restore_roberta_segmentation_sentence
from codegen_sources.model.src.model import build_model
from codegen_sources.model.src.utils import AttrDict, TREE_SITTER_ROOT

SUPPORTED_LANGUAGES = ["cpp", "java", "python"]

logger = create_logger(None, 0)



class Translator:
    def __init__(self, model_path, BPE_path):
        # reload model
        reloaded = torch.load(model_path, map_location="cpu")
        # change params of the reloaded model so that it will
        # relaod its own weights and not the MLM or DOBF pretrained model
        reloaded["params"]["reload_model"] = ",".join([model_path] * 2)
        reloaded["params"]["lgs_mapping"] = ""
        reloaded["params"]["reload_encoder_for_decoder"] = False
        self.reloaded_params = AttrDict(reloaded["params"])

        # build dictionary / update parameters
        self.dico = Dictionary(
            reloaded["dico_id2word"], reloaded["dico_word2id"], reloaded["dico_counts"]
        )
        assert self.reloaded_params.n_words == len(self.dico)
        assert self.reloaded_params.bos_index == self.dico.index(BOS_WORD)
        assert self.reloaded_params.eos_index == self.dico.index(EOS_WORD)
        assert self.reloaded_params.pad_index == self.dico.index(PAD_WORD)
        assert self.reloaded_params.unk_index == self.dico.index(UNK_WORD)
        assert self.reloaded_params.mask_index == self.dico.index(MASK_WORD)

        # build model / reload weights (in the build_model method)
        encoder, decoder = build_model(self.reloaded_params, self.dico)
        self.encoder = encoder[0]
        # self.decoder = decoder[0]
        self.encoder.cuda()
        # self.decoder.cuda()
        self.encoder.eval()
        # self.decoder.eval()

        # reload bpe
        if getattr(self.reloaded_params, "roberta_mode", False):
            self.bpe_model = RobertaBPEMode()
        else:
            self.bpe_model = FastBPEMode(
                codes=os.path.abspath(BPE_path), vocab_path=None
            )

    def translate(
        self,
        input_code,
        lang1,
        suffix1="_sa",
        suffix2="_sa",
        n=1,
        sample_temperature=None,
        device="cuda:0",
        tokenized=False,
        detokenize=True,
        max_tokens=1024,
        length_penalty=0.5,
        max_len=None,
    ):

        # Build language processors
        assert lang1 in SUPPORTED_LANGUAGES, lang1
        src_lang_processor = LangProcessor.processors[lang1](
            root_folder=TREE_SITTER_ROOT
        )
        tokenizer = src_lang_processor.tokenize_code


        lang1 += suffix1

        assert (
            lang1 in self.reloaded_params.lang2id.keys()
        ), f"{lang1} should be in {self.reloaded_params.lang2id.keys()}"

        with torch.no_grad():
            lang1_id = self.reloaded_params.lang2id[lang1]
            # lang2_id = self.reloaded_params.lang2id[lang2]

            # Convert source code to ids
            if tokenized:
                tokens = input_code.strip().split()
            else:
                tokens = [t for t in tokenizer(input_code)]
            
            
            

            #print(f"Tokenized {lang1} function:")
            #print(tokens)
            tokens = self.bpe_model.apply_bpe(" ".join(tokens)).split()
            tokens = ["</s>"] + tokens + ["</s>"]
            
            #### MODIFIED HERE --> Use the first part of functions that are too long
            ##
            if len(tokens) > 1024:
                tokens = tokens[:1024]
            input_code = " ".join(tokens)
            if max_tokens is not None and len(input_code.split()) > max_tokens:
                logger.info(
                    f"Ignoring long input sentence of size {len(input_code.split())}"
                )
                print("Input is too long...")
                return None

            # Create torch batch
            len1 = len(input_code.split())
            len1 = torch.LongTensor(1).fill_(len1).to(device)
            x1 = torch.LongTensor([self.dico.index(w) for w in input_code.split()]).to(
                device
            )[:, None]
            langs1 = x1.clone().fill_(lang1_id)

            # Encode
            enc1 = self.encoder("fwd", x=x1, lengths=len1, langs=langs1, causal=False)
            enc1 = enc1.transpose(0, 1)
            if n > 1:
                enc1 = enc1.repeat(n, 1, 1)
                len1 = len1.expand(n)


            return enc1, len1.item()




adding to path /content


In [ ]:
import time

Our class for splitting code and embedding it using the encoder as a pretrained model from transCoder.

In [ ]:
from logging import root
from codegen_sources.preprocessing.lang_processors.python_processor import PythonProcessor

class TextEncoder:

  def __init__(self,
               model_path = '/content/gdrive/MyDrive/RSS/Models/translator_transcoder_size_from_DOBF.pth', 
               BPE_path = '/content/gdrive/MyDrive/RSS/Models/', 
               parse_tree_root = "./tree-sitter-python"):

      self.p_processor = PythonProcessor(root_folder=parse_tree_root)
      self.translator = Translator(model_path, BPE_path)
  
  def get_functions(self, code):
      tokenized_code = self.p_processor.tokenize_code(code)
      functions = self.p_processor.extract_functions(tokenized_code)
      return functions[0] + functions[1]

  def process_text(self, base_code):
      # start = time.time()
      functions = self.get_functions(base_code)
      second = time.time()
      # print(second-start)
      results = []
      for index,func in enumerate(functions):
          with torch.no_grad():            
              output = self.translator.translate(func,lang1='python')
          if output is None:
              print(f"Encoding at function index {index} failed.")
          else:
              results.append(output[0].mean(dim=1).detach().cpu())
      # print(time.time()-second)
      return functions, results


In [ ]:
data = []
encoder = TextEncoder()
i = 1
times = []
for row in code.iterrows():
    start = time.time()
    id = row[1]['id']
    try:
        print('Processing code', i)
        c = row[1]['code']
        res = encoder.process_text(c)
        datapt = [id, res[0], res[1]]
        data.append(datapt)
    except:
        print('Failed on', id)
    i += 1
    times.append(time.time()-start)
    if i == 200:
      break

INFO - 04/26/22 15:11:20 - 0:09:41 - ============ Model Reloading
INFO - 04/26/22 15:11:20 - 0:09:41 - Reloading encoder from /content/gdrive/MyDrive/RSS/Models/translator_transcoder_size_from_DOBF.pth ...
WARNING - 04/26/22 15:11:22 - 0:09:43 - Lang cpp_sa matched to pretrained cpp_sa lang embedding.
WARNING - 04/26/22 15:11:22 - 0:09:43 - Lang java_sa matched to pretrained java_sa lang embedding.
WARNING - 04/26/22 15:11:22 - 0:09:43 - Lang python_sa matched to pretrained python_sa lang embedding.
INFO - 04/26/22 15:11:22 - 0:09:43 - Reloading decoders from /content/gdrive/MyDrive/RSS/Models/translator_transcoder_size_from_DOBF.pth ...
WARNING - 04/26/22 15:11:24 - 0:09:46 - Lang cpp_sa matched to pretrained cpp_sa lang embedding.
WARNING - 04/26/22 15:11:24 - 0:09:46 - Lang java_sa matched to pretrained java_sa lang embedding.
WARNING - 04/26/22 15:11:24 - 0:09:46 - Lang python_sa matched to pretrained python_sa lang embedding.
INFO - 04/26/22 15:11:24 - 0:09:46 - Number of paramete

Processing code 1
0.010053634643554688
0.05098843574523926
Processing code 2
0.024817466735839844
0.17630362510681152
Processing code 3
0.012145757675170898
0.07665824890136719
Processing code 4
0.025154590606689453
0.20233750343322754
Processing code 5
0.03563523292541504
0.2286548614501953
Processing code 6
0.027796030044555664
0.2080059051513672
Processing code 7
0.009528875350952148
0.18248486518859863
Processing code 8
0.04576683044433594
0.56313157081604
Processing code 9
0.043760061264038086
0.24843382835388184
Processing code 10
0.030311107635498047
0.03512167930603027
Processing code 11
0.002378702163696289
5.245208740234375e-05
Processing code 12
0.013867616653442383
0.015070438385009766
Processing code 13
0.034552574157714844
0.20344161987304688
Processing code 14
0.049897193908691406
0.05775332450866699
Processing code 15
0.014428853988647461
0.16242361068725586
Processing code 16
0.04957008361816406
0.010039567947387695
Processing code 17
0.052684783935546875
0.01443195343

In [ ]:
np.array(times).mean()

0.15854143855547664

In [ ]:
(785*38*.1534)/60

76.26536666666668

In [ ]:
codedf = pd.DataFrame(data, columns=['id', 'functions', 'representations'])
codedf.to_pickle('/content/gdrive/MyDrive/RSS/Data/func-rep-small.pt')
codedf.head()

,id,functions,representations
0,1811.02182,[def main ( config ) : NEW_LINE INDENT from da...,"[[[tensor(0.6232), tensor(-0.2805), tensor(0.3..."
1,1811.02182,"[def check_config_used ( config , target_sourc...","[[[tensor(0.6108), tensor(-0.2560), tensor(0.1..."
2,1811.02182,"[def __init__ ( self , batch_size , paired = F...","[[[tensor(0.8430), tensor(0.1034), tensor(0.30..."
3,1811.02182,[def weights_init ( m ) : NEW_LINE INDENT clas...,"[[[tensor(0.6790), tensor(-0.0221), tensor(0.1..."
4,1811.02182,[def weights_init ( m ) : NEW_LINE INDENT clas...,"[[[tensor(0.6790), tensor(-0.0221), tensor(0.1..."


In [ ]:
codedf.iloc[0].representations[0].shape

torch.Size([1, 1024])

In [ ]:
from google.colab import files
codedf.to_csv('code-func-rep.csv')
files.download('code-func-rep.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
encoder = TextEncoder()
res = encoder.process_text(code.iloc[0]['code'])
res[0], res[1]

torch.Size([1, 1024])

In [ ]:
res[1].shape

torch.Size([1, 1024])

In [ ]:
demo_alg = algs.iloc[10]['alg']

In [ ]:
%pip install pylatexenc

     |████████████████████████████████| 162 kB 33.9 MB/s 
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136835 sha256=bdc314832bc7b072c872a8a8cec310738991c4eee20ee913c04a6f9d5492ea15
  Stored in directory: /root/.cache/pip/wheels/f1/8a/f5/33ee79d4473eb201b519fa40f989b842e373237395a3421f52
Successfully built pylatexenc


In [ ]:
from pylatexenc.latex2text import LatexNodes2Text

latex = demo_alg
text = LatexNodes2Text().latex_to_text(latex)
# import re
print(demo_alg)
# re.split('[\s{}]',demo_alg)
print(word_tokenize(text))
# word_tokenize(demo_alg)

# sentences = sent_tokenize(text)
# examples = []
# for sentence in sentences:
#     sentence = "".join(char for char in sentence if char not in punctuation)
#     sentence = "".join(char for char in sentence if not char.isdigit())
#     sentence = sentence.lower()
#     tokens = word_tokenize(sentence)
#     examples.append(tokens)
# return examples

# tokenize("Why, hello there! What's your name?")

[t]
\caption{Hypergraph marginalization for HMMs and PCFGs}
\label{fig:marg-hmm-pcfg}
\begin{minipage}[t]{0.45\textwidth}
%\small
\begin{algorithmic}
\STATE {[\textit{HMM - Backward}]}
\FOR {$t \leftarrow (t+1)$ in right-to-left order}
\FOR {$z_{t+1} \in \mcL$}
\STATE $[\beta_{t+1}]_{z_{t+1}} = [\alpha_{t+1}]_{z_{t+1}}$
\ENDFOR
\STATE $\alpha_t \stackrel{+}{\gets} \Psi_t \beta_{t+1}$
\ENDFOR
\STATE \textbf{return} $\alpha_0^\top \mathbf{1}$
\end{algorithmic}
\end{minipage}
\vspace{0pt}
\begin{minipage}[t]{0.50\textwidth}
%\small
\begin{algorithmic} 
\STATE {[\textit{PCFG - CKY}]}
\FOR {$(i,k) \leftarrow (i,j), (j,k)$ in span-size order}
\FOR {$z_1,z_2 \in \mcL_{i,j}\times\mcL_{j,k}$}
\STATE $[\beta_{i,j,k}]_{(z_1,z_2)} = [\alpha_{i,j}]_{z_1}[\alpha_{j,k}]_{z_2}$
\ENDFOR
\STATE $\alpha_{i,k} \stackrel{+}{\gets} \Psi\beta_{i,j,k}$
\ENDFOR
\STATE \textbf{return} $\alpha_{1,T}^\top \mathbf{1}$
\end{algorithmic}
\end{minipage}

['[', 't', ']', 'Hypergraph', 'marginalization', 'for', 'HMMs',

In [ ]:
uni_toks = {i for i in word_tokenize(text)}

In [ ]:
uni_toks

{'(',
 ')',
 '+Ψ_t',
 ',',
 '-',
 '0.45',
 '0.50',
 '=',
 'Backward',
 'CKY',
 'HMM',
 'HMMs',
 'Hypergraph',
 'PCFG',
 'PCFGs',
 'T^⊤1',
 '[',
 ']',
 '_',
 '_z_1',
 '_z_2',
 '_z_t+1',
 'and',
 'for',
 'i',
 'in',
 'j',
 'j×_j',
 'k',
 'k+Ψβ_i',
 'marginalization',
 'order',
 'return',
 'right-to-left',
 'span-size',
 't',
 't+1',
 'z_1',
 'z_2',
 'z_t+1∈',
 'α_0^⊤1',
 'α_1',
 'α_i',
 'α_j',
 'α_t',
 'α_t+1',
 'β_i',
 'β_t+1',
 '←',
 '∈_i'}

In [ ]:
import transformers
import torch
from transformers import BertTokenizer
from transformers import DistilBertTokenizer

print(torch.__version__)
print(transformers.__version__)

# THERE IS A UNICODE character between the , and '' (specifically \U+200D\U+200D\U+200D\U+200D)
sentence = ' '.join(uni_toks)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
# tokenizer = BertTokenizer.from_pretrained(
#             "bert-base-cased",
#             do_lower_case=False,
#             use_fast=False
#         )

print(uni_toks)
print(tokenizer.tokenize(sentence))

# special_tokens_dict = {'additional_special_tokens': ['\uf0b7']}
# num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
# test_words = ['crazy', 'character', '\uf0b7']
# tokenizer(test_words, is_split_into_words=True)

1.10.0+cu111
4.18.0
{'-', 'order', 'for', 'α_0^⊤1', 'PCFGs', ',', 'span-size', 'k', '_z_2', 'z_t+1∈', 't', '=', 'j×_j', 'j', 't+1', 'Hypergraph', '∈_i', 'α_i', 'and', 'CKY', ']', 'α_1', 'k+Ψβ_i', '0.50', 'marginalization', 'z_1', 'β_i', '[', 'T^⊤1', 'α_j', ')', 'in', 'HMMs', 'Backward', 'right-to-left', '←', '+Ψ_t', '_z_t+1', 'HMM', 'α_t', 'β_t+1', 'PCFG', '_z_1', 'z_2', 'α_t+1', 'return', 'i', '_', '(', '0.45'}
['-', 'order', 'for', 'α', '_', '0', '^', '[UNK]', 'pc', '##f', '##gs', ',', 'span', '-', 'size', 'k', '_', 'z', '_', '2', 'z', '_', 't', '+', '1', '##∈', 't', '=', 'j', '##×', '_', 'j', 'j', 't', '+', '1', 'hyper', '##graph', '∈', '_', 'i', 'α', '_', 'i', 'and', 'ck', '##y', ']', 'α', '_', '1', 'k', '+', 'ψ', '##β', '_', 'i', '0', '.', '50', 'marginal', '##ization', 'z', '_', '1', 'β', '_', 'i', '[', 't', '^', '[UNK]', 'α', '_', 'j', ')', 'in', 'hmm', '##s', 'backward', 'right', '-', 'to', '-', 'left', '←', '+', 'ψ', '_', 't', '_', 'z', '_', 't', '+', '1', 'hmm', 'α', '_', 't'